##### Copyright 2020 The TensorFlow Hub Authors.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classify text with BERT

This fine-tuned BERT model is trained to classify tweets into 3 labels, 0 = "hate speech", 1 = "offensive language", 2 = "neither". 

Trained on https://github.com/t-davidson/hate-speech-and-offensive-language

testing dataset from https://github.com/hate-alert/HateXplain

## Setup


In [2]:
# A dependency of the preprocessing for BERT inputs
#!pip install -q -U "tensorflow-text==2.8.*"
#!pip install -q tf-models-official==2.7.0

## Loading in packages and initilizing constants

In [3]:
import os
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


from datasets import load_dataset
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

device = '/GPU:0' if tf.test.is_gpu_available()else "/CPU:0"

tf.get_logger().setLevel('ERROR')
MODEL_NAME = "hate_bert"
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 12
seed = 42
test_train_ratio = 0.1

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


## Loading in datasets

hate_data holds the train and validaiton dataset from the hatespeech dataset

train_ds, val_ds, and test_ds are batched and shuffled tf.data.Dataset 

In [9]:
dataset = load_dataset("hatexplain", split="validation")

class_names = ["hate speech", "normal", "offensive"]

dataset = dataset.map(lambda e: {'sent_tokens' : ' '.join(e['post_tokens'])})
dataset = dataset.map(lambda e: {'label' : int(np.median(e['annotators']['label']))})

dataset_tensor = dataset.with_format(type='tensorflow', columns=['sent_tokens'])

input_tensor = []
for i in range(0,len(dataset_tensor)):
    input_tensor += [dataset_tensor[i]['sent_tokens']]

Reusing dataset hatexplain (C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)
Loading cached processed dataset at C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249\cache-d5dd4819f594bfe0.arrow
Loading cached processed dataset at C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249\cache-512fd663cee324b1.arrow


[1, 1, 1]
1


## Loading the BERT model


In [5]:
model_name = 'hate_speech_bert_L-12_H-768_A-12'
saved_model_path = './{}'.format(model_name.replace('/', '_'))


reloaded_model = hub.load(saved_model_path)
print(type(reloaded_model))

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


In [10]:
outputs = []
with tf.device(device):
    for x in range(0,len(dataset_tensor), batch_size):
        o = reloaded_model(dataset_tensor['sent_tokens'][x:x+batch_size])
        outputs += (dataset['label'][x:x+batch_size], o) 
print(outputs)

NameError: name 'reloaded_model' is not defined